# Ingredients transformation
    - Last change : 12/05/2021
    - Ingredients processing to get new features
    - Exported csv as 'data_training_multi_pnns.csv'

### Short Description

Usage of a json file with ingredients information (text, taxonomy id, percent estimate) for 200K+ products (mainly french) in the off products database to create 1 column per N most frequent ingredients with percent estimate (or median if missing). We chose the 450 most frequent ingredients.

## Imports

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
ing = pd.read_json(r'C:\Users\Antoine\Documents\GitHub\openfoodfacts-ai\ai-emlyon\robotoff_predictions\fr_products_with_ingredients_235211.json')
ing.shape

(235211, 3)

In [3]:
ing = ing.rename(columns={"_id": "code"})
ing

,code,ingredients,ingredients_text
0,0.000000e+00,"[{'text': 'Farine de blé', 'vegan': 'yes', 've...","Farine de blé, huile de palme, amidon de tapio..."
1,1.000000e+02,[{'text': 'eau graines de téguments de moutard...,eau graines de téguments de moutarde vinaigre ...
2,1.199000e+03,"[{'ingredients': [{'id': 'en:e316', 'text': 'é...","antioxydant : érythorbate de sodium, colorant ..."
3,1.663000e+03,"[{'percent_min': 20, 'vegan': 'no', 'text': 'L...","Lait entier, sucre, amidon de maïs, cacao, Aga..."
4,2.264000e+03,"[{'text': 'baguette Poite vin Pain baguette', ...","baguette Poite vin Pain baguette 50,6%: farine..."
...,...,...,...
235206,9.990334e+07,"[{'percent_estimate': 50, 'rank': 1, 'percent_...",Jus de fruits à base de jus concentrés et puré...
235207,9.991111e+12,"[{'vegetarian': 'yes', 'text': 'Pomme', 'vegan...","Pomme 71 %, poire 26 %, sucre 4%, antioxydant ..."
235208,9.999201e+12,"[{'id': 'en:sugar', 'percent_max': 100, 'perce...","sucre,beurre de cacao,poudre de jait.entier,fa..."
235209,9.999864e+12,[{'text': 'BIOCOOP BORDEAUX LAC distribue par ...,BIOCOOP BORDEAUX LAC distribue par les eleveur...


In [4]:
#Drop text
ing = ing.drop(columns=['ingredients_text'])

In [5]:
#Read df with pnns
df = pd.read_csv(r'C:\Users\Antoine\Coding Bootcamp\Open Food Facts\df_multi_pnns_v2.csv', low_memory=False)

In [6]:
#Copy
df_copy = df.copy(deep=True)

In [7]:
#Custom columns dict
from eml.datanavig import off_columns_dict
cols = off_columns_dict.copy()

In [8]:
#Changing code type
df['code'] = df.code.astype(float)

In [9]:
#Merge original data with ingredients json
df_with_ing = pd.merge(df, ing, on=['code'], how='right')
df_with_ing = df_with_ing.dropna(subset=['url'])
df_with_ing.shape

(167890, 8)

In [10]:
df_with_ing.head(3)

,pnns_groups_1,pnns_groups_2,pnns_groups_3,pnns_groups_4,code,url,product_name,ingredients
1,fat and sauces,dressings and sauces,dressings and sauces,dressings and sauces,100.0,http://world-en.openfoodfacts.org/product/0000...,moutarde au moût de raisin,[{'text': 'eau graines de téguments de moutard...
8,beverages,sweetened beverages,sweetened beverages,artificially sweetened beverages,5.0,http://world-en.openfoodfacts.org/product/0000...,Nectar d'abricot,"[{'percent_max': 50, 'text': 'Jus', 'id': 'en:..."
9,composite foods,one dish meals,legumes,legumes,5.0,http://world-en.openfoodfacts.org/product/05/r...,Ratatouille à la provençale,"[{'percent_max': 50, 'text': 'Jus', 'id': 'en:..."


## Navigation into structured ingredients file

In [11]:
#Set df_test
ingredients = df_with_ing['ingredients']
df_test = ingredients.sample(1000, random_state=42)

In [12]:
#Save all ingredients names
id_ingredients = []
for ingre_list in df_with_ing.ingredients:
    for ingre_dict in ingre_list:
        id_ingredients.append(ingre_dict['text'].replace('_','').replace('-','').strip('').lower())

In [13]:
len(set(id_ingredients))

203089

In [14]:
#Check count by ingredient
ing_list = pd.DataFrame(data=np.array(id_ingredients), columns=['ingredient'])
ing_list = pd.DataFrame(ing_list['ingredient'].value_counts())
ing_list = ing_list.reset_index()
ing_list.columns = ['ingredient','count']
ing_list.loc[ing_list['count'] > 1000]

,ingredient,count
0,sel,105822
1,sucre,84031
2,eau,77957
3,farine de blé,28033
4,émulsifiant,23861
...,...,...
309,viande de dinde,1010
310,chocolat en poudre,1010
311,coprah,1010
312,boyau naturel de porc,1005


In [15]:
#400 most frequent ingredients
ing_to_add = list(ing_list['ingredient'][:400])

## Function to create ingredients columns

In [16]:
def get_ingredients_columns(df, ingredients_list):
    #Expensive in ram & cpu
    """
    Create a new column for each item in ing_list
    and fill with 1 if the row contains ingredient and 0 if not"""
    data = df.copy(deep=True) #make a copy of df
    for i in ingredients_list: data[i] = 0 #create 0 columns, 1 per ingredient
    for ingre_list, index in zip(data.ingredients, data.index): #loop over df rows
        for ingre_dict in ingre_list: #loop continue in the dicts in each row
            val = ingre_dict['text'] #get ingredient text
            val_clean = val.replace('_','').replace('-','').strip('').lower()
            if val_clean in ingredients_list: #check if val is in columns added
                try:
                    data.loc[index,val_clean] = ingre_dict['percent_estimate'] #if yes, replace by estimate
                except:
                    pnns_val = data.loc[index,'pnns_groups_1']
                    val_median = data[val_clean].loc[data['pnns_groups_1'] == pnns_val].median()
                    data.loc[index,val_clean] = val_median
    return data #return modified dataframe

### Test function

In [17]:
df_test = df_with_ing.sample(10000, random_state=42)
df_test = pd.DataFrame(df_test)
ing_test = []
for ingre_list in df_test.ingredients:
    for ingre_dict in ingre_list:
        ing_test.append(ingre_dict['text'].replace('_','').replace('-','').strip('').lower())
to_fill = ing_test[:75]
modif = get_ingredients_columns(df=df_test, ingredients_list= to_fill)
modif.describe().T

,count,mean,std,min,25%,50%,75%,max
code,10000.0,1.036044e+39,1.036044e+41,24.000000,3.245414e+12,3.375644e+12,3.760153e+12,1.036044e+43
levure,10000.0,1.088693e-01,8.229927e-01,-0.168750,0.000000e+00,0.000000e+00,0.000000e+00,1.750000e+01
gluten de blé,10000.0,6.286489e-02,1.041397e+00,-0.675000,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01
farine de fèves,10000.0,8.560311e-03,4.147444e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.995000e+01
émulsifiant,10000.0,1.295042e-01,8.989792e-01,-0.485079,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+01
antioxydant,10000.0,1.476444e-01,1.277573e+00,-3.230903,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01
acidifiants,10000.0,4.591070e-02,6.503942e-01,-0.100000,0.000000e+00,0.000000e+00,0.000000e+00,2.250000e+01
dextrose,10000.0,2.795992e-01,2.521741e+00,-0.962500,0.000000e+00,0.000000e+00,0.000000e+00,9.925000e+01
amidon transformé de pomme de terre,10000.0,2.064675e-03,6.551941e-02,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.125000e+00
stabilisant,10000.0,8.220027e-02,8.330085e-01,-0.104167,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+01


### Apply func to full data

In [18]:
#*450 most frequent ingredients
ing_to_add = list(ing_list['ingredient'][:450])

In [20]:
%%time
df_filled = get_ingredients_columns(df_with_ing, ing_to_add)
df_filled.shape

Wall time: 1h 54min 29s


(167890, 458)

## Save data to a new csv

In [22]:
df_filled.to_csv(r'C:\Users\Antoine\Coding Bootcamp\Open Food Facts\data_training_multi_pnns.csv', index = False, header=True)